## Introduction to Project

Business Case - A new streaming company, LiveWire, is beginning to understand the market that has the majority of its viewers utilizing streaming systems such as: Netflix, Amazon Prime, and Hulu. LiveWire needs the help of a Data Scientist to understand what movies are best to include in its library that are appealing to a wide variety of viewers and needs a recommendation system like these other large brand names have to keep the consumer engaged and interested with their product.

Questions to answer throughout the modeling project:
    1. What are the top 5 genres that should be focused on when adding to the library?
    2. How can a similar user help the recommendation system suggest movies to a new user?
    3. What are the top ten movies of all time based off this DataFrame being utilized?

In [2]:
!conda install surprise

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - surprise

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




## Importing Libraries

In [3]:
# import libraries
import numpy as np
import pandas as pd

from surprise import Dataset, Reader
from surprise import KNNBaseline, SVD, SVDpp, NMF
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split

## EDA

In [4]:
df1 = pd.read_csv(('movies.csv'), index_col = 0)
df1.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [5]:
df1.describe()

,title,genres
count,58098,58098
unique,58020,1643
top,Escape Room (2017),Drama
freq,2,8402


In [6]:
df2 = pd.read_csv(('ratings.csv'), index_col=0)
df2.set_index('movieId')
df2

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,movieId,rating,timestamp
userId,,,
1,307,3.5,1256677221
1,481,3.5,1256677456
1,1091,1.5,1256677471
1,1257,4.5,1256677460
1,1449,4.5,1256677264
...,...,...,...
283228,8542,4.5,1379882795
283228,8712,4.5,1379882751
283228,34405,4.5,1379882889


In [7]:
# Creating one df with the two seperate df's
df = df2.merge(df1, right_index=True, left_on='movieId')
df = df.reset_index()

In [8]:
df

,userId,movieId,rating,timestamp,title,genres
0,1,307,3.5,1256677221,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
1,6,307,4.0,832059248,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
2,56,307,4.0,1383625728,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,71,307,5.0,1257795414,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
4,84,307,3.0,999055519,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
...,...,...,...,...,...,...
27753439,282403,167894,1.0,1524243885,Stranglehold (1994),Action
27753440,282732,161572,3.5,1504408070,The Great Houdini (1976),Drama
27753441,283000,117857,3.5,1417317969,Hotline (2014),Documentary
27753442,283000,133409,3.5,1431539331,Barnum! (1986),(no genres listed)


In [9]:
# Creating df based on movie ID's as the index
avg_rating = df.groupby('movieId').mean()
avg_rating

,userId,rating,timestamp
movieId,,,
1,141939.237859,3.886649,1.128094e+09
2,142395.293962,3.246583,1.105961e+09
3,140371.877575,3.173981,9.692933e+08
4,140527.990632,2.874540,9.405874e+08
5,141254.322735,3.077291,9.970812e+08
...,...,...,...
193876,103565.000000,3.000000,1.537875e+09
193878,176871.000000,2.000000,1.537875e+09
193880,81710.000000,2.000000,1.537886e+09


In [10]:
# Adding the title into the new df and dropping irrelevant column 'timestamp'
df_avg_rtng = avg_rating.merge(df1, right_index=True, left_on='movieId')
df_avg_rtng = df_avg_rtng.drop(columns='timestamp')
df_avg_rtng

,userId,rating,title,genres
movieId,,,,
1,141939.237859,3.886649,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,142395.293962,3.246583,Jumanji (1995),Adventure|Children|Fantasy
3,140371.877575,3.173981,Grumpier Old Men (1995),Comedy|Romance
4,140527.990632,2.874540,Waiting to Exhale (1995),Comedy|Drama|Romance
5,141254.322735,3.077291,Father of the Bride Part II (1995),Comedy
...,...,...,...,...
193876,103565.000000,3.000000,The Great Glinka (1946),(no genres listed)
193878,176871.000000,2.000000,Les tribulations d'une caissière (2011),Comedy
193880,81710.000000,2.000000,Her Name Was Mumu (2016),Drama


In [16]:
dummies = df_avg_rtng['genres'].str.get_dummies(sep='|')
dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53889 entries, 1 to 193886
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   (no genres listed)  53889 non-null  int64
 1   Action              53889 non-null  int64
 2   Adventure           53889 non-null  int64
 3   Animation           53889 non-null  int64
 4   Children            53889 non-null  int64
 5   Comedy              53889 non-null  int64
 6   Crime               53889 non-null  int64
 7   Documentary         53889 non-null  int64
 8   Drama               53889 non-null  int64
 9   Fantasy             53889 non-null  int64
 10  Film-Noir           53889 non-null  int64
 11  Horror              53889 non-null  int64
 12  IMAX                53889 non-null  int64
 13  Musical             53889 non-null  int64
 14  Mystery             53889 non-null  int64
 15  Romance             53889 non-null  int64
 16  Sci-Fi              53889 non-null  int

In [12]:
df_avg_rtng.drop_duplicates(inplace=True)
df_avg_rtng.shape

(53889, 4)

In [15]:
df_dummies = df_avg_rtng.merge(dummies, right_index=True, left_on='genres')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
print(df_avg_rtng.info())
print(df_avg_rtng.describe())

In [ ]:
# Sorting df with ratings 5-0
five_to_zero_rating = df_avg_rtng.sort_values(['rating'], ascending=False)
five_to_zero_rating

In [ ]:
five_to_zero_rating['genres'].split(sep='|')
five_to_zero_rating

In [ ]:
five_to_zero_rating['genres'].astype(str)

In [ ]:
five_to_zero_rating['genres'].describe()

In [ ]:
five_to_zero_rating['genres'].value_counts().head(10)

In [ ]:
# Condition
# top_votes = (five_to_zero_rating['rating'] >= '4.0')
avg_rating_genre = df.groupby('genres').mean()
avg_rating_genre

## Creating a Recommendation Model

In [ ]:
# def model_accuracy(model):
#     model.fit(smote_x, smote_y)
    
    
#     y_train_pred = model.predict(X_train)
#     print(round(accuracy_score(y_train, y_train_pred)*100,2),'%')
#     print(confusion_matrix(y_train, y_train_pred))
    
#     y_pred = model.predict(X_test)
#     print(round(accuracy_score(y_test, y_pred)*100,2),'%')
#     print(confusion_matrix(y_test, y_pred))

In [ ]:
KNNB = KNNBaseline()
model_accuracy(KNNB)

In [ ]:
SVD = SVD()
model_accuracy(SVD)

In [ ]:
SVDpp = SVDpp()
model_accuracy(SVDpp)

In [ ]:
NMF = NMF()
model_accuracy(NMF)